In [417]:
# ejecucion
proyecto_en = "PC"  # kaggle - PC

In [418]:
def get_folder_out(project):
    if project == "kaggle":
        return "/kaggle/output/working/"
    if project == "PC":
        return os.sep.join(os.getcwd().split(os.sep)[:-1]) + os.sep + 'kaggle/output/working/'

def get_folder_int(project):
    if project == "kaggle":
        return "/kaggle/input"
    if project == "PC":
        return os.sep.join(os.getcwd().split(os.sep)[:-1]) + os.sep + 'kaggle/input/musicnet-dataset'

In [419]:
FOLDER_OUT = get_folder_out(proyecto_en) # kaggle - PC

In [420]:
# CONFIG

MINIMA_VARIANA_EXPLICADA = 0.93 # se debe definir porque este valor
TIME_SPLIT = 1 # falta definir porque este split de 2

# Folder
FOLDER_SAVE_NORMALIZADOR_PCA = FOLDER_OUT + 'normalizador_pca.pkl'
FOLDER_MODEL = FOLDER_OUT + "randomforest.pkl"

FOLDER_TRAIN_DATA = FOLDER_OUT + "train_data.csv"
FOLDER_TRAIN_LABEL = FOLDER_OUT + "train_label.csv"
FOLDER_TEST_DATA = FOLDER_OUT + "test_data.csv"
FOLDER_TEST_LABEL = FOLDER_OUT + "test_label.csv"

FOLDER_TAGS_LABEL = FOLDER_OUT + "tags_label.csv"
# Model
SPLIT_DATA_TRAIN = 0.2



In [421]:
import os
from distutils.version import StrictVersion

# preprocesando los datos
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# para guardar el preprocesador de datos
import pickle
from scipy.io.wavfile import write

# Modelado
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Evaluacion
import sklearn.metrics as metrics

# quitar alertas innecesarias
import warnings

warnings.filterwarnings('ignore')

In [422]:
# librerias personalizadas

In [423]:
import scipy.io.wavfile as wav
import pandas as pd


class Corte_audio():
    SEGUNDOS_CORTE = 5

    def __init__(self, config_time: int = 5, save_audios: str = None):
        self.SEGUNDOS_CORTE = config_time
        self.save_audios = save_audios

    def __hallar_instrumentos(self, corte_start, corte_end, instrument, start_time):
        acumulador = list()
        for (i, time_start) in enumerate(start_time):
            if corte_end >= start_time[i] >= corte_start:
                # print(start_time[i], end_time[i], instrument[i])
                if not instrument[i] in acumulador:
                    acumulador.append(instrument[i])
            else:
                if time_start > corte_end:
                    break
        return acumulador

    def ___save_data(self, data_save, file):
        df = pd.DataFrame(data=data_save, columns=['instrument'], index=None, dtype=None, copy=False)
        df.to_csv(file, index=False)

    def split_data(self, path_wav: str = "", path_csv: str = "", tag: str = "dat"):
        (rate, sig) = wav.read(path_wav)
        labels = pd.read_csv(path_csv)

        instrument = labels['instrument']
        start_time = labels['start_time']

        pivote = 0
        corte = rate * self.SEGUNDOS_CORTE

        muestras = list()
        instrumentos = list()
        for _ in range(round(len(sig) / corte)):
            corte_start, corte_end = pivote, pivote + corte
            data = sig[corte_start:corte_end]
            muestras.append(data)
            instrumentos.append(self.__hallar_instrumentos(corte_start, corte_end, instrument, start_time))
            pivote += corte

        if pivote < len(sig):
            data = sig[pivote:]
            muestras.append(data)
            instrumentos.append(self.__hallar_instrumentos(pivote, len(sig), instrument, start_time))

        if self.save_audios is not None:
            for i, data in enumerate(muestras):
                wav.write(self.save_audios + f"{tag}_{i}.wav", rate, data)
                self.___save_data(instrumentos[i], self.save_audios + f"{tag}_{i}.csv")

        return muestras, instrumentos, rate

In [424]:
import librosa
vers_required = "0.7.2"
if StrictVersion(librosa.__version__) < StrictVersion(vers_required):
    print("Error: minimum librosa vers: {}, current vers {}".format(vers_required, librosa.__version__))
    !pip install librosa===0.7.2 --force-reinstall

print("current vers", librosa.__version__)

current vers 0.9.2


In [425]:
import matplotlib.pyplot as plt
import librosa.display
import numpy as np

class ProcessAudio(object):
    data = None

    def __init__(self, sr:int = 44100):
        self.mfcc = None
        self.zcr = None
        self.rolloff = None
        self.spec_bw = None
        self.spec_cent = None
        self.rmse = None
        self.chroma_stft = None
        self.sr = sr

    def set_data(self, data):
        self.data = data

    def display_waveform(self):
        if self.data is None:
            return None
        # display waveform
        plt.figure(figsize=(14, 5))
        librosa.display.waveplot(self.data , sr=self.sr)

    def get_croma(self):
        if self.data is None:
            return None
        self.chroma_stft = librosa.feature.chroma_stft(y=self.data, sr=self.sr)
        return self.chroma_stft

    def get_rmse(self):
        if self.data is None:
            return None
        self.rmse = librosa.feature.rms(y=self.data)
        return self.rmse

    def get_centroide_espectral(self):
        """centroide espectral"""
        if self.data is None:
            return None
        self.spec_cent = librosa.feature.spectral_centroid(y=self.data, sr=self.sr)
        return self.spec_cent

    def get_ancho_banda_espectral(self):
        if self.data is None:
            return None
        self.spec_bw = librosa.feature.spectral_bandwidth(y=self.data, sr=self.sr)
        return self.spec_bw

    def get_rolloff(self):
        """tambien conocido como reduccion espectral"""
        if self.data is None:
            return None
        self.rolloff = librosa.feature.spectral_rolloff(y=self.data, sr=self.sr)
        return self.rolloff

    def get_cruce_por_cero(self):
        if self.data is None:
            return None
        self.zcr = librosa.feature.zero_crossing_rate(self.data)
        return self.zcr

    def get_mfcc(self):
        if self.data is None:
            return None
        self.mfcc = librosa.feature.mfcc(y=self.data, sr=self.sr)
        return self.mfcc

    def get_all(self, i:int) -> list:
        if self.data is None:
            return []

        self.get_croma()
        self.get_rmse()
        self.get_centroide_espectral()
        self.get_ancho_banda_espectral()
        self.get_rolloff()
        self.get_cruce_por_cero()
        self.get_mfcc()

        data_compresed = f'train{i} {np.mean(self.chroma_stft)} {np.mean(self.rmse)} {np.mean(self.spec_cent)} {np.mean(self.spec_bw)} {np.mean(self.rolloff)} {np.mean(self.zcr)}'
        for e in self.mfcc:
            data_compresed += f' {np.mean(e)}'

        return data_compresed.split()

In [426]:
import nlpaug.augmenter.audio as naa
import librosa.display as librosa_display


class AudioAugmentation:
    graph: bool = False
    print_middleware: bool = False

    def __init__(self, audio_file, graph=False, save: str = None):
        self.audio_file = audio_file
        self.graph = graph
        self.save = save
        if isinstance(audio_file, str):
            print("Cargando archivo de audio")
            self.name_file = audio_file.split(".")[-2].split("/")[-1]
            self.data, self.rate = self.__read_audio_file(self.audio_file)
        elif isinstance(audio_file, tuple):
            #print("Audio file is a tuple")
            self.data, self.rate, self.name_file = audio_file
            #print(len(self.data), self.rate, self.name_file)

    def middleware_in(atributo1: str = "", ):
        def _middleware_in(f):
            def wrapper(self, *args, **kwargs):
                # ejecucion antes de ejecutar la funcion
                if self.print_middleware:
                    print(f"Middleware in {f.__name__} ({atributo1})")

                # ejecucion de la funcion
                ejecucion_ok = True
                try:
                    salida = f(self, *args, **kwargs)
                except Exception as e:
                    ejecucion_ok = False
                    salida = e
                    print(f"Error en la ejecucion de la funcion {f.__name__}: {e}")

                # ejecucion despues de ejecutar la funcion
                if ejecucion_ok:
                    if self.graph:
                        self.plot_audio(salida)

                    if self.save is not None:
                        try:
                            if isinstance(salida, list):
                                if self.print_middleware:
                                    print(salida)
                                salida = np.array(salida)
                            self.write_audio_file(self.save + self.name_file + "_" + f"{f.__name__}.wav", salida, self.rate)
                        except Exception as e:
                            if self.print_middleware:
                                print(f"Error al guardar el archivo: {e}")
                            print(type(salida))

                return salida

            return wrapper

        return _middleware_in

    def plot_audio(self, data2=None):
        librosa_display.waveshow(self.data, sr=self.rate, alpha=0.5)
        if data2 is not None:
            librosa_display.waveshow(data2, sr=self.rate, color='r', alpha=0.25)

        plt.tight_layout()
        plt.show()

    @middleware_in(atributo1="loudness_f")
    def loudness(self):
        aug = naa.LoudnessAug()
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

    @middleware_in(atributo1="mask_f")
    def add_mask(self):
        aug = naa.MaskAug(sampling_rate=self.rate, mask_with_noise=False)
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

    @middleware_in(atributo1="pitch_f")
    def pitch(self, fact=(2,3)):
        aug = naa.PitchAug(sampling_rate=self.rate, factor=fact)
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

    @middleware_in(atributo1="original")
    def get_original(self):
        return self.data

    @middleware_in(atributo1="crop")
    def add_crop(self, porcentaje: float = 0.5):
        crop = naa.CropAug(sampling_rate=self.rate)
        augmented_data = crop.augment(self.data)[0]
        return augmented_data

    @staticmethod
    def __read_audio_file(file_path):
        input_length = 16000
        data, rate = librosa.core.load(file_path)
        if len(data) > input_length:
            data = data[:input_length]
        else:
            data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        return data, rate

    @staticmethod
    def write_audio_file(file, data, sample_rate):
        write(file, sample_rate, data)

    @staticmethod
    def plot_time_series(data):
        fig = plt.figure(figsize=(14, 8))
        plt.title('Raw wave ')
        plt.ylabel('Amplitude')
        plt.plot(np.linspace(0, 1, len(data)), data)
        plt.show()

    @middleware_in(atributo1="ruido")
    def add_noise(self, factor_ruido=0.005):
        noise = np.random.randn(len(self.data))
        data_noise = self.data + factor_ruido * noise
        data_noise = data_noise.astype(type(self.data[0]))  # Cast back to same
        return data_noise

    @middleware_in(atributo1="ruido2")
    def add_noise2(self):
        aug = naa.NoiseAug()
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

    @middleware_in(atributo1="shift")
    def shift(self):
        return np.roll(self.data, 1600)

    @middleware_in(atributo1="stretch")
    def stretch(self, rate_stretch=1.0):
        input_length = 16000
        data = librosa.effects.time_stretch(self.data, rate=rate_stretch)
        if len(data) > input_length:
            data = data[:input_length]
        else:
            data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        return data

    @middleware_in(atributo1="speed")
    def speed(self):
        aug = naa.SpeedAug()
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

    @middleware_in(atributo1="normalizador")
    def normalizer(self):
        aug = naa.NormalizeAug(method='minmax')
        augmented_data = aug.augment(self.data)[0]
        return augmented_data

    @middleware_in(atributo1="polarizador")
    def polarizer(self):
        aug = naa.PolarityInverseAug()
        augmented_data = aug.augment(self.data)[0]
        return augmented_data


class Audio_K(AudioAugmentation):

    def __init__(self, file_path, label: list, save: str = None, grafica: bool = False):
        super().__init__(file_path, save=save, graph=grafica)
        self.label = label

    def aumentar(self):
        all_data = [
            self.get_original(),
            self.add_noise(factor_ruido=0.05),
            self.add_noise2(),
            self.stretch(rate_stretch=0.8),
            self.shift(),
            self.add_crop(),
            self.loudness(),
            self.speed(),
            self.normalizer(),
            self.polarizer()
        ]

        all_label = [self.label for _ in range(len(all_data))]
        return all_data, all_label



In [427]:
from functools import wraps
from time import time


def count_elapsed_time(f):
    @wraps(f)
    def cronometro(*args, **kwargs):
        t_inicial = time()  # tomo la hora antes de ejecutar la funcion
        salida = f(*args, **kwargs)
        t_final = time()  # tomo la hora despues de ejecutar la funcion
        print('Tiempo transcurrido (en segundos): {}'.format(t_final - t_inicial))
        return salida

    return cronometro

In [428]:
# iniciador de clases

pca_pipe = make_pipeline(StandardScaler(), PCA(MINIMA_VARIANA_EXPLICADA))

In [429]:
# organizando los datos en train y test (data y label para cada uno)

TRAIN = {}
TEST = {}

@count_elapsed_time
def crear_diccionarios_train_test():
    for dirname, _, filenames in os.walk(get_folder_int(proyecto_en)):
        for filename in filenames:
            file = os.path.join(dirname, filename)
            name_file, extension = filename.split(".")

            if extension == "csv" or extension == "wav":
                if dirname.find("train")>0:
                    if name_file not in TRAIN:
                        TRAIN[name_file] = {}
                    if filename.find("csv")>0:
                        TRAIN[name_file]['label'] = file
                    else:
                        TRAIN[name_file]['data'] = file
                else:
                    if name_file not in TEST:
                        TEST[name_file] = {}
                    if filename.find("csv")>0:
                        TEST[name_file]['label'] = file
                    else:
                        TEST[name_file]['data'] = file
crear_diccionarios_train_test()

Tiempo transcurrido (en segundos): 0.009542226791381836


In [430]:
TODOS_LABEL = {}

In [431]:
def save_data(data_save, file):
    df = pd.DataFrame(data=data_save)
    df.to_csv(file)

In [432]:
mean = lambda lst: int((sum(lst) / len(lst)) * 100) / 100

def calcular_porcentajes_aciertos(y_f, y_t):
    verdaderos = dict()
    falsos = dict()
    for j in range(y_f.shape[1]):
        verdaderos[j] = 0
        falsos[j] = 0

    for i in range(y_f.shape[0]):
        for j in range(y_f.shape[1]):
            if y_f[i][j] == y_t[i][j]:
                verdaderos[j] += 1
            else:
                falsos[j] += 1

    for j in range(y_f.shape[1]):
        # y_final.shape[1] -> 100%
        # verdaderos[j]    -> X
        verdaderos[j] = int(verdaderos[j] * 100 / y_f.shape[0])
        falsos[j] = int(falsos[j] * 100 / y_f.shape[0])

    return verdaderos, falsos, str(mean([v for i, v in verdaderos.items()])) + "%"

In [433]:
# Abriendo cada audio, a cada uno se le aplica el split y a cada audio resultante se le extraen caracteristicas


@count_elapsed_time
def files_to_data(diccionario_datos, buscar_todos_label: bool = False, save_new_data: str = None, data_aumentation: bool = False):
    global TODOS_LABEL
    conteo = 0

    DATA = []
    LABEL = []

    rate = 0

    cortador = Corte_audio(config_time=TIME_SPLIT, save_audios=save_new_data)

    for name_file, value in diccionario_datos.items():
        if 'data' in value and 'label' in value:
            conteo += 1
            print(name_file, end=" - ")
            if conteo >= 20:
                print()
                conteo = 0

            muestras_wav, instrumentos, rate = cortador.split_data(value['data'], value['label'])

            if data_aumentation and buscar_todos_label:
                new_muestras = []
                new_instruments = []
                for id_audio, dat in enumerate(muestras_wav):
                    data_audio_para_amplificar= (dat, rate, "data_" + str(id_audio))
                    label_audio_para_amplificar = instrumentos[id_audio]
                    data_amplificada, label_amplificada = Audio_K(data_audio_para_amplificar, label=label_audio_para_amplificar).aumentar()
                    #print(len(data_amplificada))
                    for id_new_audio, new_audio in enumerate(data_amplificada):
                        new_data_add = new_audio
                        new_label_add = label_amplificada[id_new_audio]

                        ok = True
                        try:
                            cte = 110250
                            if (TIME_SPLIT*rate) > len(new_data_add):
                                new_data_add += [0 for _ in range(cte*TIME_SPLIT)]
                            if (TIME_SPLIT*rate) < len(new_data_add):
                                new_data_add = new_data_add[:TIME_SPLIT*cte]
                        except Exception as e:
                            ok = False

                        if ok:
                            new_muestras.append(new_data_add)
                            new_instruments.append(new_label_add)

                muestras_wav, instrumentos = new_muestras, new_instruments

            processAudio = ProcessAudio(rate)
            for id_audio, dat in enumerate(muestras_wav):
                processAudio.set_data(dat)
                # print(len(dat), end = ">")
                ok = True
                caracteristicas = None
                try:
                    caracteristicas = processAudio.get_all(id_audio)  # Extrayendo caracteristicas audios, salen 26 caracteristicas
                except:
                    ok = False
                if ok:
                    DATA.append(caracteristicas[1:])
                    LABEL.append(instrumentos[id_audio])

    # buscando todos los label
    if buscar_todos_label:
        for lab in LABEL:
            for la in lab:
                if la not in TODOS_LABEL:
                    TODOS_LABEL[la] = 0
        TODOS_LABEL = tuple(sorted(TODOS_LABEL))

    # expandiendo los label a su respectivo vector de etiquetas
    for j, lab in enumerate(LABEL):
        new_label = [0 for _ in TODOS_LABEL]
        for la in lab:
            for i, l in enumerate(TODOS_LABEL):
                if l == la:
                    new_label[i] = 1
        LABEL[j] = new_label


    # convirtiendo a numpy los datos
    DATA = np.array(DATA, dtype=float)
    LABEL = np.array(LABEL, dtype=float)

    return DATA, LABEL

In [434]:
DATA, LABEL = files_to_data(TRAIN, buscar_todos_label=True, data_aumentation=True)

1727 - 1728 - 1729 - 1730 - 1733 - 1734 - 1735 - 1739 - 1742 - 1749 - 1750 - 1751 - 1752 - 1755 - 1756 - 1757 - 1758 - 1760 - 1763 - 1765 - 
1766 - 1768 - 1771 - 1772 - 1773 - 1775 - 1776 - 1777 - 1788 - 1789 - 1790 - 1791 - 1792 - 1793 - 1805 - 1807 - 1811 - 1812 - 1817 - 1818 - 
1822 - 1824 - 1828 - 1829 - 1835 - 1859 - 1872 - 1873 - 1876 - 1893 - 1916 - 1918 - 1919 - 1922 - 1923 - 1931 - 1932 - 2075 - 2076 - 2077 - 
2078 - 2079 - 2080 - 2081 - 2082 - 2083 - 2104 - 2105 - 2112 - 2113 - 2114 - 2116 - 2117 - 2118 - 2119 - 2127 - 2138 - 2140 - 2147 - 2148 - 
2149 - 2150 - 2151 - 2154 - 2155 - 2156 - 2157 - 2158 - 2159 - 2160 - 2161 - 2166 - 2167 - 2168 - 2169 - 1764 - 1813 - 1933 - 2131 - 2177 - 
2213 - 2239 - 2300 - 2342 - 2381 - 2422 - 2478 - 2510 - 2557 - 2594 - 2178 - 2179 - 2180 - 2186 - 2194 - 2195 - 2196 - 2198 - 2200 - 2201 - 
2202 - 2203 - 2204 - 2207 - 2208 - 2209 - 2210 - 2211 - 2212 - 2214 - 2215 - 2217 - 2218 - 2219 - 2220 - 2221 - 2222 - 2224 - 2225 - 2227 - 
2228 - 2229 -

In [435]:
print(len(DATA), len(DATA[0]))
print(len(LABEL), len(LABEL[0]))

888543 26
888543 11


In [436]:
save_data([TODOS_LABEL], FOLDER_TAGS_LABEL)

In [437]:
# Normalizando y aplicando PCA
pca_pipe.fit(DATA)
pickle.dump(pca_pipe, open(FOLDER_SAVE_NORMALIZADOR_PCA,'wb'))

In [438]:
def preparar_datos_para_modelo(datos):
    normalizador_pca = pickle.load(open(FOLDER_SAVE_NORMALIZADOR_PCA, 'rb'))
    x_for_model = normalizador_pca.transform(X=datos)
    return x_for_model

In [439]:
DATA = preparar_datos_para_modelo(DATA)

In [440]:
print(f"Original vector size: {len(DATA[0])} -> New vector size {len(DATA[0])} ({int(MINIMA_VARIANA_EXPLICADA*100)}% información mantenida)")

Original vector size: 15 -> New vector size 15 (93% información mantenida)


In [441]:
# Guardando datos
save_data(DATA, FOLDER_TRAIN_DATA)
save_data(LABEL, FOLDER_TRAIN_LABEL)

In [442]:
X_train, X_valid, y_train, y_valid = train_test_split(DATA, LABEL, test_size=0.1)  # 0.2

In [443]:
seed = 1
grid = GridSearchCV(
          estimator = RandomForestClassifier(),
          param_grid={},
          cv = KFold(n_splits=10, shuffle=True, random_state=seed)
        )

In [444]:
# Entrenando
grid.fit(X_train, y_train)
model = grid.best_estimator_
pickle.dump(model, open(FOLDER_MODEL, 'wb'))

In [445]:
y_final = model.predict(X_valid)

In [446]:
matrix_confusion_personalizada = calcular_porcentajes_aciertos(y_final, LABEL)[2]
print("matrix_confusion_personalizada:", matrix_confusion_personalizada)

matrix_confusion_personalizada: 84.81%


In [447]:
print("ACC", metrics.accuracy_score(y_valid, y_final))
print("PREC", metrics.precision_score(y_valid, y_final, average='micro'))

ACC 0.8338979235833661
PREC 0.9392451877376425


# Testear el modelo

In [448]:
# Cargando el modelo guardado
model = pickle.load(open(FOLDER_MODEL, 'rb'))

In [449]:
# Leyendo datos de testeo y calculando vector de caracteristicas
if not os.path.exists(FOLDER_OUT + "new_data/"):
    os.makedirs(FOLDER_OUT + "new_data/")

DATA, LABEL = files_to_data(TEST, save_new_data=str(FOLDER_OUT + "new_data/"))

1759 - 1819 - 2106 - 2191 - 2298 - 2303 - 2382 - 2416 - 2556 - 2628 - Tiempo transcurrido (en segundos): 66.27676653862


In [450]:
DATA_REDU = preparar_datos_para_modelo(DATA)

In [451]:
print(f"Original vector size: {len(DATA[0])} -> New vector size {len(DATA_REDU[0])} ({int(MINIMA_VARIANA_EXPLICADA*100)}% información mantenida)")

Original vector size: 26 -> New vector size 15 (93% información mantenida)


In [452]:
# Guardando datos
save_data(DATA_REDU, FOLDER_TEST_DATA)
save_data(LABEL, FOLDER_TEST_LABEL)

In [453]:
y_final = model.predict(DATA_REDU)

In [454]:
matrix_confusion_personalizada = calcular_porcentajes_aciertos(y_final, LABEL)[2]
print("matrix_confusion_personalizada:", matrix_confusion_personalizada)

matrix_confusion_personalizada: 95.0%


In [455]:
print("ACC", metrics.accuracy_score(LABEL, y_final))
print("PREC", metrics.precision_score(LABEL, y_final, average='micro'))

ACC 0.705050505050505
PREC 0.865924276169265


## Testo unitario

In [456]:
path_label_test_unit = FOLDER_OUT + "new_data/dat_0.csv"
path_data_test_unit = FOLDER_OUT + "new_data/dat_0.wav"

In [457]:
def expandir_tags_out(original_tag, all_tags):
    LABEL = original_tag.get("instrument").to_numpy()

    # expandiendo los label al array de tags entrenado
    new_label = [0 for _ in all_tags]
    for la in LABEL:
        for i, l in enumerate(all_tags):
            if l == la:
                new_label[i] = 1
    LABEL = new_label
    return LABEL

@count_elapsed_time
def predecir_archivo_unitario(data_wav, rate):
    processAudio = ProcessAudio(rate)
    processAudio.set_data(data_wav)
    DATA = processAudio.get_all("0")[1:]  # Extrayendo caracteristicas audios, salen 26 caracteristicas
    DATA_REDU = preparar_datos_para_modelo([DATA])

    # Cargando el modelo guardado
    model = pickle.load(open(FOLDER_MODEL, 'rb'))
    y_final = model.predict(DATA_REDU)

    return y_final

In [458]:
(DATA, rate) = librosa.core.load(path_data_test_unit)

y_final = predecir_archivo_unitario(DATA, rate)

Tiempo transcurrido (en segundos): 71.18626976013184


In [459]:
LABEL = pd.read_csv(path_label_test_unit, header=0)
LABEL = expandir_tags_out(LABEL, TODOS_LABEL)

matrix_confusion_personalizada = calcular_porcentajes_aciertos(y_final, [LABEL])[2]
print("matrix_confusion_personalizada:", matrix_confusion_personalizada)

print("ACC", metrics.accuracy_score([LABEL], y_final))
print("PREC", metrics.precision_score([LABEL], y_final, average='micro'))

matrix_confusion_personalizada: 90.9%
ACC 0.0
PREC 1.0
